In [1]:
import os
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input, concatenate
from tensorflow.keras.models import Model

print(tf.test.is_built_with_cuda())

##data.py
##read the images

imsize = 512       #image size
bridge_layer = 512 # bridge layer fileters in U-NET 
num_filters = [32, 64, 128, 256] # Filters in U-net
training = 1 # program should only carry out testting

def load_data(path, split=0.1):
    images = sorted(glob(os.path.join(path,"./BCSS-master/images/*")))
    masks = sorted(glob(os.path.join(path,"./BCSS-master/masks_tumor/*")))

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(imsize, imsize))
    x = x/255.0
    return x

def read_image_test(path):
    #path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(imsize, imsize))
    x = x/255.0
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (imsize, imsize))
    x = x/255.0
#     x = np.expand_dims(x, axis=-1)
    return x

def read_mask_test(path):
    #path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (imsize, imsize))
    x = x/255.0
#     x = np.expand_dims(x, axis=-1)
    return x

def myfun(img):
    img = np.squeeze(img)

    return img

def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float64, tf.float64])
    x.set_shape([imsize, imsize, 3])
    y.set_shape([imsize, imsize, 3])
    return x, y

def tf_dataset(x, y, batch=16):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    return dataset

#model.py
#building the u-net arch



def conv_block(x, num_filters):
    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, (3, 3), padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def build_unet_model():
    size = imsize
    
    inputs = Input((size, size, 3))

    skip_x = []
    x = inputs
    ## Encoder
    for f in num_filters:
        x = conv_block(x, f)
        skip_x.append(x)
        x = MaxPool2D((2, 2))(x)

    ## Bridge
    x = conv_block(x, bridge_layer)

    num_filters.reverse()
    skip_x.reverse()
    ## Decoder
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        xs = skip_x[i]
        x = concatenate([x, xs])
        x = conv_block(x, f)

    ## Output
    x = Conv2D(3, (3, 3), padding="same")(x)
    x = Activation("sigmoid")(x)
    return Model(inputs, x)

##Train.py

def build_segmentation_model():
    num_filters = [32, 64, 128, 256]
    size = imsize
    
    inputs_1 = Input((size, size, 3))
    x = inputs_1
    
    # Encoder
    skip_connections = []
    for i, f in enumerate(num_filters):
        x = Conv2D(f, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(f, (3, 3), activation='relu', padding='same')(x)
        skip_connections.append(x)
        x = MaxPooling2D((2, 2))(x)

    # Bridge
    x = Conv2D(num_filters[-1], (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(num_filters[-1], (3, 3), activation='relu', padding='same')(x)

    # Expansive Path
    skip_connections = skip_connections[::-1]
    num_filters = num_filters[::-1]
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2))(x)
        x = Concatenate()([x, skip_connections[i]])
        x = Conv2D(f, (3, 3), activation='relu', padding='same')(x)
        x = Conv2D(f, (3, 3), activation='relu', padding='same')(x)

    # Output
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)

    x = skip_connections[0]

    model = Model(inputs=inputs_1, outputs=x, name='segmentation_model')
    return model

def build_combined_model():
    inputs = Input((imsize, imsize, 3))
    unet_output = build_unet_model()(inputs)
    segmentation_output = build_segmentation_model()(unet_output)
    model = Model(inputs=inputs, outputs=segmentation_output, name='combined_model')
    return model

# def build_combined_model():
#     unet_model = build_unet_model()
#     segmentation_model = build_segmentation_model()

#     # Get the output of the UNet model
#     unet_output = unet_model.output

#     # Pass the UNet output to the input of the segmentation model
#     additional_input = Input(shape=(imsize, imsize, 1))
#     segmentation_input = segmentation_model([unet_output, additional_input])

#     combined_model = Model(inputs=[unet_model.input, additional_input], outputs=segmentation_input)
#     return combined_model

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)



False


In [3]:
## Hyperparameters
batch = 8
lr = 1e-3
epochs = 2

path = "."
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)
train_dataset = tf_dataset(train_x, train_y, batch=batch)
valid_dataset = tf_dataset(valid_x, valid_y, batch=batch)
test_dataset = tf_dataset(test_x, test_y, batch=batch)

r_model = build_unet_model()
print("r_model")
r_model.summary()

s_model = build_segmentation_model()
print("s_model")
s_model.summary()

if os.path.isfile("./BCSS-master/files/model.h5"):
    with CustomObjectScope({'iou': iou}):
        combined_model = tf.keras.models.load_model("./BCSS-master/files/model.h5")
else:
    combined_model = build_combined_model()		
print("Model Buid....")
combined_model.summary()



r_model
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                     

In [4]:
opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
combined_model.compile(loss="mse", optimizer=opt, metrics=metrics)

callbacks = [
        ModelCheckpoint("./BCSS-master/files/model.h5"),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
        CSVLogger("./BCSS-master/files/data.csv"),
        TensorBoard(),
        #EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
    ]


train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch
test_steps = len(test_x)//batch
if len(train_x) % batch != 0:
    train_steps += 1
if len(valid_x) % batch != 0:
    valid_steps += 1
if len(test_x) % batch != 0:
    test_steps += 1  
print("Training ....")
#     print(train_dataset)
#     print(valid_dataset)
#     print(train_steps)
#     print(valid_steps)
#     print(callbacks)
if training == 1:
    combined_model.fit(train_dataset,steps_per_epoch=train_steps, batch_size=batch, epochs=epochs, validation_data=valid_dataset,callbacks=callbacks)
    # combined_model.fit(train_dataset,
    #     validation_data=valid_dataset,
    #     epochs=epochs,
    #     steps_per_epoch=train_steps,
    #     validation_steps=valid_steps,
    #     callbacks=callbacks)
evalcallbacks = [
    CSVLogger("./BCSS-master/files/eval.csv")
    ]

combined_model.evaluate(test_dataset, steps=test_steps, callbacks=evalcallbacks)


Training ....
Epoch 1/2


ValueError: in user code:

    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\aadit\anaconda3\envs\DL\lib\site-packages\keras\engine\compile_utils.py", line 201, in __call__
        loss_value = l

In [ ]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
    x = read_image_test(x)
    y = read_mask_test(y)
    y_pred = combined_model.predict(np.expand_dims(x, axis=0))
    #cv2.imwrite("./results/"+str(i)+".png", y_pred)
    h, w, _ = x.shape
    white_line = np.ones((h, 5, 3)) * 255.0
    #print("Printing sizes\n")
        #print(mask_parse(y_pred).shape)
        #print(mask_parse(y).shape)
        # print(x.shape)
        # print(myfun(y_pred).shape)
    all_images = [
        x * 255.0, white_line,
        y*255.0, white_line,
        myfun(y_pred) * 255.0
    ]
    image = np.concatenate(all_images, axis=1)
    cv2.imwrite(f"results/{i}.png", image)
print(type(image))